In [8]:
# Import statements:
import pandas as pd
import numpy as np
import torch
import time

# Hugging face import:
from transformers import pipeline

In [11]:
# Saving zero-shot classification: 
theme_pipeline = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli")

# Saving sentiment pipeline
sentiment_pipeline = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [2]:
# Loading in data:
#df = pd.read_csv("/Users/amaribauer/Desktop/A_ML/FinalProject/okcupid_profiles.csv")
df = pd.read_csv("/Users/rohitkandala/Desktop/UChicago/Academic Quarters/2023-24/Winter 2024/CAPP 30255/Project/okcupid_profiles.csv")

In [50]:
# Essay dataframe:
essays_df = df.loc[:, ["essay0", "essay1", "essay2", "essay3", "essay4", 
                   "essay5", "essay6", "essay7", "essay8", "essay9"]]
essays_df = essays_df.astype(str)

# Essay0 dataframe only "about me":
essay0_df = df.loc[:, ["essay0"]]
essay0_df = essay0_df.dropna(subset=['essay0'])

In [ ]:
# Write cleaning function that removes stopwords, etc - potentially already done by pretrained model. 

Proposed Additions to labels: travel, drinking, drugs, kids
Proposed Removals: teen, enthusiastic, time periods, avid, miscellaneous, rock, sci-fi, favorite, novelty

In [41]:
# Setting labels. Think of thse as our classes:
candidate_labels = ['TV hits', 'enthusiastic', 'movies', 'music',
          'comedies', 'food', 'teen', 'everything', 'drama', 'time periods', 
          'avid', 'miscellaneous', 'rock', 'music', 'sci-fi', 'favorite', 
          'novelty', 'books']

In [51]:
# Create a sample of 10 observations:
sampled_df = essay0_df.sample(10)

In [52]:
# Checking indexes of dataframe that are included in the sample:
print(sampled_df.index)

Index([34118, 18708, 31958, 27741, 29665, 41977, 4789, 26894, 41198, 31154], dtype='int64')


In [53]:
sampled_df.head()

,essay0
34118,here are some facts with a high degree of trut...
18708,i recently graduated with a masters in account...
31958,i likes colors and patterns. i enjoy spending ...
27741,"the words that best describe me are active, vi..."
29665,it's really hard to describe myself in 100 cha...


In [75]:
# Function helps with classification & making sure that tensors are all the
# same length:

def classify_essay(essay):

    # Perform zero-shot classification
    output = theme_pipeline(essay, candidate_labels)
    
    # Create a dictionary mapping labels to scores:
    score_dict = {label: score for label, score in zip(
        output['labels'], output['scores'])}
    
    # Ensure all candidate labels have a score, set to 0 if missing:
    for label in candidate_labels:
        if label not in score_dict:
            score_dict[label] = 0.0
    
    # Convert the dictionary to a pandas Series:
    return pd.Series(score_dict)

In [76]:
# Applying zero-shot classification to sampled dataframe & creating a dataframe:
results = sampled_df['essay0'].apply(classify_essay)

In [80]:
# Seeing dataframe:
results.head(10)

,enthusiastic,avid,food,music,miscellaneous,novelty,favorite,time periods,teen,rock,drama,everything,sci-fi,TV hits,movies,books,comedies
34118,0.279540,0.154115,0.137217,0.113735,0.062601,0.039644,0.020007,0.017548,0.013915,0.009514,0.009237,0.008404,0.005144,0.004558,0.004208,0.003518,0.003361
18708,0.157826,0.434765,0.022420,0.010858,0.053393,0.052949,0.028648,0.058519,0.022478,0.015866,0.018481,0.026516,0.011439,0.014050,0.009090,0.041317,0.010528
31958,0.264436,0.156601,0.060781,0.046317,0.065510,0.038110,0.016730,0.057300,0.012433,0.012400,0.005459,0.011714,0.004129,0.004217,0.002705,0.003341,0.191501
27741,0.470504,0.191120,0.010947,0.031321,0.053294,0.021188,0.102768,0.018213,0.011227,0.019705,0.003613,0.013623,0.004863,0.006505,0.002312,0.002842,0.004635
29665,0.154645,0.181510,0.006840,0.024050,0.179129,0.148447,0.050645,0.029104,0.061061,0.036226,0.010375,0.019723,0.016278,0.018012,0.010524,0.013225,0.016157
41977,0.057058,0.103458,0.021358,0.071587,0.185273,0.088847,0.057465,0.070695,0.026750,0.034804,0.037131,0.024514,0.011995,0.026255,0.016279,0.069078,0.025867
4789,0.074426,0.179151,0.072553,0.016739,0.052094,0.129600,0.035143,0.166758,0.055707,0.040257,0.041743,0.013736,0.012366,0.021353,0.028182,0.017417,0.026036
26894,0.044932,0.173299,0.007152,0.007227,0.335098,0.206065,0.046044,0.038007,0.018977,0.011658,0.055244,0.012508,0.011739,0.005333,0.004653,0.005898,0.008938
41198,0.306287,0.131433,0.003635,0.006974,0.135380,0.137794,0.026101,0.087784,0.022274,0.047342,0.014976,0.025141,0.010097,0.012470,0.007432,0.005886,0.012019
31154,0.501350,0.366823,0.001421,0.004004,0.037566,0.018263,0.022966,0.004276,0.006174,0.006903,0.003260,0.004134,0.005597,0.002447,0.003841,0.005490,0.001480


In [87]:
# Function to get sentiment for each essay0:
def get_sentiment(essay):

    # Run sentiment analysis on the essay text
    sentiment_result = sentiment_pipeline(essay)

    # Extract the sentiment label and score
    label = sentiment_result[0]['label']
    score = sentiment_result[0]['score']

    # Return a dictionary with label and score
    return {'sentiment_label': label, 'sentiment_score': score}

In [ ]:
# 

In [22]:
#TO DO: BUILD OUT LOGIC TO PROCESS MORE ROWS

# Apply the sentiment_pipeline function to each essay in the results Series and transpose the resulting DataFrame
sentiments_df = pd.DataFrame(results.apply(lambda essay: [results.name, 
        essay['sequence'], essay['labels'][0], sentiment_pipeline(essay['sequence'])]).tolist(
        ), columns=['Index', 'Sequence', 'Label', 'Sentiment']).transpose()

# Combine all columns into a single row
long_col_names =  which_essays + ['theme1', 'theme2','theme3', 'theme4', 'theme5'] + ['sent1','sent2','sent3','sent4','sent5']
sentiments_combined = pd.DataFrame(sentiments_df.values.flatten().reshape(1, -1))

Important: Our end result here, sentiments_long, is a pandas df with the essay responses, the top themes, and sentiment analysis response for five essays. Our output for the eventual final classifier would a pandas dataframe with this information for every profile. An alternative implementation could be to include each essay, top theme, and sentiment into a tuple, and then we'd only have five columns per person instead. 

In [23]:
sentiments_long = sentiments_combined.drop(sentiments_combined.columns[:5], axis = 1)
sentiments_long.columns = long_col_names
sentiments_long

,essay0,essay1,essay2,essay3,essay5,theme1,theme2,theme3,theme4,theme5,sent1,sent2,sent3,sent4,sent5
0,about me: i would love to think that i was so...,currently working as an international agent fo...,making people laugh. ranting about a good salt...,"the way i look. i am a six foot half asian, ha...",food. water. cell phone. shelter.,favorite,books,enthusiastic,miscellaneous,food,"[{'label': 'POSITIVE', 'score': 0.997051715850...","[{'label': 'NEGATIVE', 'score': 0.997330904006...","[{'label': 'POSITIVE', 'score': 0.998056292533...","[{'label': 'NEGATIVE', 'score': 0.853669345378...","[{'label': 'NEGATIVE', 'score': 0.740077495574..."


Next steps:
- Saving probabilities into a dataset after computing all of them, consult with Ethan on the topic names prior to running everything

----

Generative AI portion:

- import gpt-2 or some other model
- fine tune model on the essays in the dataset
- generate text in response to the input essay

link with example of gpt2: https://huggingface.co/openai-community/gpt2

- How do we incorporate higher matches to train the model? ## Ask the TA? 
    - One model is most likely feasible, but the best we have brain stormed is what if we do multiple models (one for each label)
        - When user inputs text, classifier identifies top topic -> generate text in response with corresponding topic model

-----

Matching methodology:

- One proposal: lets take the probabilities for each label between two individuals and compute the distance between the probabilities across ALL categories and aggregate them, that is the "compatiability index". 
    - Only conduct matching search for those who are compatiable, sexuality-wise
        - Goal is to limit the amount of cross-computation
    - Compute compatiability index ONLY between people who have the same top topic AND sexualtiy

-----

